In [69]:
import requests
import pandas as pd 
import numpy as np

In [2]:
# https://opendata.dc.gov/datasets/70392a096a8e431381f1f692aaa06afd_24?geometry=-77.446%2C38.808%2C-76.579%2C38.995

def download_file(url):
    local_filename = "data/accidents.csv"
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=16384): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    return local_filename

url = "https://opendata.arcgis.com/datasets/70392a096a8e431381f1f692aaa06afd_24.csv"
fileName = download_file(url)

In [55]:
fileName = "data/accidents.csv"
usecolsall = ["X","Y","OBJECTID","CRIMEID","CCN","REPORTDATE","ROUTEID","MEASURE","OFFSET","STREETSEGID","ROADWAYSEGID","FROMDATE","TODATE","MARID","ADDRESS","LATITUDE","LONGITUDE","XCOORD","YCOORD","WARD","EVENTID","MAR_ADDRESS","MAR_SCORE","MAJORINJURIES_BICYCLIST","MINORINJURIES_BICYCLIST","UNKNOWNINJURIES_BICYCLIST","FATAL_BICYCLIST","MAJORINJURIES_DRIVER","MINORINJURIES_DRIVER","UNKNOWNINJURIES_DRIVER","FATAL_DRIVER","MAJORINJURIES_PEDESTRIAN","MINORINJURIES_PEDESTRIAN","UNKNOWNINJURIES_PEDESTRIAN","FATAL_PEDESTRIAN","TOTAL_VEHICLES","TOTAL_BICYCLES","TOTAL_PEDESTRIANS","PEDESTRIANSIMPAIRED","BICYCLISTSIMPAIRED","DRIVERSIMPAIRED","TOTAL_TAXIS","TOTAL_GOVERNMENT","SPEEDING_INVOLVED","NEARESTINTROUTEID","NEARESTINTSTREETNAME","OFFINTERSECTION","INTAPPROACHDIRECTION","LOCATIONERROR","LASTUPDATEDATE","MPDLATITUDE","MPDLONGITUDE","MPDGEOX","MPDGEOY","BLOCKKEY"]
accidentData = pd.read_csv(fileName, usecols=usecolsall)
accidentData['FROMDATE'] = pd.to_datetime(accidentData['FROMDATE']) 
accidentData = accidentData[accidentData['FROMDATE'] > "2016-12-31"]
smallData['FROMDATE'] = pd.to_datetime(smallData['FROMDATE']) 
smallData = smallData[smallData['FROMDATE'] > "2016-12-31"]

In [56]:
print(list(accidentData))

['X', 'Y', 'OBJECTID', 'CRIMEID', 'CCN', 'REPORTDATE', 'ROUTEID', 'MEASURE', 'OFFSET', 'STREETSEGID', 'ROADWAYSEGID', 'FROMDATE', 'TODATE', 'MARID', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'WARD', 'EVENTID', 'MAR_ADDRESS', 'MAR_SCORE', 'MAJORINJURIES_BICYCLIST', 'MINORINJURIES_BICYCLIST', 'UNKNOWNINJURIES_BICYCLIST', 'FATAL_BICYCLIST', 'MAJORINJURIES_DRIVER', 'MINORINJURIES_DRIVER', 'UNKNOWNINJURIES_DRIVER', 'FATAL_DRIVER', 'MAJORINJURIES_PEDESTRIAN', 'MINORINJURIES_PEDESTRIAN', 'UNKNOWNINJURIES_PEDESTRIAN', 'FATAL_PEDESTRIAN', 'TOTAL_VEHICLES', 'TOTAL_BICYCLES', 'TOTAL_PEDESTRIANS', 'PEDESTRIANSIMPAIRED', 'BICYCLISTSIMPAIRED', 'DRIVERSIMPAIRED', 'TOTAL_TAXIS', 'TOTAL_GOVERNMENT', 'SPEEDING_INVOLVED', 'NEARESTINTROUTEID', 'NEARESTINTSTREETNAME', 'OFFINTERSECTION', 'INTAPPROACHDIRECTION', 'LOCATIONERROR', 'LASTUPDATEDATE', 'MPDLATITUDE', 'MPDLONGITUDE', 'MPDGEOX', 'MPDGEOY', 'BLOCKKEY']


In [57]:
smallData = accidentData[['TOTAL_BICYCLES','TOTAL_VEHICLES','TOTAL_PEDESTRIANS','LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'FATAL_BICYCLIST', 'MPDLATITUDE', 'MPDLONGITUDE', 'FROMDATE']]

In [70]:
# get only accidents that include a bike
bikeData = smallData[smallData['TOTAL_BICYCLES'] > 0]

# get only accidents that include a bike and a pedestrian
bikePedData = smallData[(smallData['TOTAL_BICYCLES'] > 0) & (smallData['TOTAL_PEDESTRIANS'] > 0) & (smallData['TOTAL_VEHICLES'] == 0)]

# get only accidents that include a bike and a vehicles
bikeVehData = smallData[(smallData['TOTAL_BICYCLES'] > 0) & (smallData['TOTAL_VEHICLES'] > 0)]

In [71]:
# save them to csvs
bikeData.to_csv("data/bikes.csv")
bikePedData.to_csv("data/bikePedData.csv")
bikeVehData.to_csv("data/bikeVehData.csv")

In [66]:
# plotting all bike accidents
dcMap = folium.Map(location=[38.9072, -77.0369], zoom_start=13)

colorsList = ['red', 'blue', 'black', 'purple', 'orange', 'pink', 'green']
def getColor(ind):
    colorLen = len(colorsList)
    colorNum = ind%colorLen
    color = colorsList[colorNum]
    return color
    
counter = 0
for i in accPoints:
    if (clusters[counter] != -1):
        folium.CircleMarker(location=[i[0], i[1]], 
                            radius = .5,
                            color = getColor(clusters[counter])).add_to(dcMap)
    counter = counter  + 1
dcMap

In [10]:
# https://opendata.dc.gov/datasets/294e062cdf2c48d5b9cbc374d9709bc0_2/data